<a href="https://colab.research.google.com/github/aned0227/matlab-pushup-code/blob/main/mrc_commission_tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install XlsxWriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 4.6 MB/s eta 0:00:00


In [ ]:

# Importing libraries and packages:
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn import metrics
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import calendar
import re

import os


In [ ]:

# Initialize a dictionary to store the DataFrames
dfs = {}

# Get the list of all files in the current directory
files = [f for f in os.listdir() if f.endswith('.csv')]  # Only CSVs

# Sort files by month
month_order = list(calendar.month_abbr)[1:]
files.sort(key=lambda x: month_order.index(re.match(r"([a-zA-Z]+)", x, re.I).group(0)[:3].capitalize()))

# Iterate over the sorted list of files and process each CSV file
for file in files:

    # Read the CSV file
    df = pd.read_csv(file)

    # Adding 'Notes' to data frame
    df['Notes'] = pd.Series(dtype='float64')

    # Possibly needed, but has not been in use as of 09/01/23
    df['Brand'].replace({'Boost': 'Tn', 'Virgin': 'BTN'}, inplace=True)

    mask = (df['Total Commissions to Date'] != 0) & (df['Deactivation Date'].isnull())& ~df['Base Plan'].str.contains('ACP', case=False)

    filtered_df = df[mask]

    condensed_df = filtered_df.loc[mask, ['Phone Number','Notes','Brand', '1st MRC Payment','2nd MRC Payment','3rd MRC Payment','4th MRC Payment']]

    # Sort by '1st MRC Payment in ascending order'
    condensed_df = condensed_df.sort_values(by="1st MRC Payment")

    # Store the DataFrame in the dictionary
    dfs[file] = condensed_df

# Accessing each DataFrame using the file name as the key
for file_name, dataframe in dfs.items():
    print(f"File: {file_name}")
    print(dataframe.head())  # Print the first 5 rows of each DataFrame, as an example


File: jan20240304.csv
      Phone Number  Notes Brand 1st MRC Payment 2nd MRC Payment  \
0   (562)-318-6112    NaN   BTN      01/03/2024      02/03/2024   
31  (562)-329-7547    NaN   BTN      01/03/2024      02/03/2024   
14  (562)-551-1863    NaN   BTN      01/03/2024      02/03/2024   
38  (626)-466-8383    NaN   BTN      01/04/2024      02/04/2024   
3   (626)-466-8386    NaN   BTN      01/04/2024      02/04/2024   

    3rd MRC Payment  4th MRC Payment  
0               NaN              NaN  
31              NaN              NaN  
14              NaN              NaN  
38              NaN              NaN  
3               NaN              NaN  
File: feb20240304.csv
      Phone Number  Notes Brand 1st MRC Payment  2nd MRC Payment  \
3   (562)-332-7361    NaN   BTN      02/02/2024              NaN   
7   (562)-309-3316    NaN   BTN      02/04/2024              NaN   
14  (562)-273-0682    NaN   BTN      02/05/2024              NaN   
0   (562)-328-6744    NaN   BTN      02/06/2024

In [ ]:

# def used to manipulate dataframe to create, manipulate and store modified dataframe
def MRC(target_month, df):
    processed_dfs = {}

    '''second MR is next'''
    mask = (df['2nd MRC Payment'].isnull()) & (df['3rd MRC Payment'].isnull())
    mrc2_df = df.loc[mask, ['Phone Number','Notes','Brand', '1st MRC Payment','2nd MRC Payment']]
    mrc2_df['1st MRC Payment'] = pd.to_datetime(mrc2_df['1st MRC Payment'], format='%m/%d/%Y')
    mrc2_df['2nd MRC Payment'] = mrc2_df['1st MRC Payment'] + pd.Timedelta(days=48) # adding 48 days to 1st mrc to determine 2nd mr
    mrc2_df['2nd MRC Payment'] = mrc2_df['2nd MRC Payment'].dt.strftime('%m/%d/%Y')
    mrc2_df['1st MRC Payment'] = mrc2_df['1st MRC Payment'].dt.strftime('%m/%d/%Y')

    ''' third MR is next '''
    mask = (df['2nd MRC Payment'].notnull()) & (df['3rd MRC Payment'].isnull())
    mrc3_df = df.loc[mask, ['Phone Number','Notes','Brand', '1st MRC Payment','2nd MRC Payment', '3rd MRC Payment']]
    mrc3_df['1st MRC Payment'] = pd.to_datetime(mrc3_df['1st MRC Payment'], format='%m/%d/%Y')
    mrc3_df['3rd MRC Payment'] = mrc3_df['1st MRC Payment'] + pd.Timedelta(days=96) # adding 96 days to 1st mrc to determine 3rd mr
    mrc3_df['3rd MRC Payment'] = mrc3_df['3rd MRC Payment'].dt.strftime('%m/%d/%Y')
    mrc3_df['1st MRC Payment'] = mrc3_df['1st MRC Payment'].dt.strftime('%m/%d/%Y')

    ''' fourth MR is next '''
    mask = (df['3rd MRC Payment'].notnull()) & (df['4th MRC Payment'].isnull())
    mrc4_df = df.loc[mask, ['Phone Number','Notes','Brand', '1st MRC Payment','2nd MRC Payment', '3rd MRC Payment', '4th MRC Payment']]
    mrc4_df['1st MRC Payment'] = pd.to_datetime(mrc4_df['1st MRC Payment'], format='%m/%d/%Y')
    mrc4_df['4th MRC Payment'] = mrc4_df['1st MRC Payment'] + pd.Timedelta(days=144) # adding 144 days to 1st mrc to determine 4th mr
    mrc4_df['4th MRC Payment'] = mrc4_df['4th MRC Payment'].dt.strftime('%m/%d/%Y')
    mrc4_df['1st MRC Payment'] = mrc4_df['1st MRC Payment'].dt.strftime('%m/%d/%Y')

    processed_dfs[f'{target_month}_2nd MRC'] = mrc2_df
    processed_dfs[f'{target_month}_3rd MRC'] = mrc3_df
    processed_dfs[f'{target_month}_4th MRC'] = mrc4_df

    return processed_dfs

In [ ]:

import calendar
import re

# Define the Excel writer with the desired Excel file name
with pd.ExcelWriter('combined_mrc_sheets.xlsx', engine='xlsxwriter') as writer:
    # Iterate over the dictionary of input data frames.
    for file_name, dataframe in dfs.items():

        # Extract month abbreviation and year from file_name
        # Assuming filenames are like 'jul20230915.csv' or 'july20230915.csv' (where month is followed by numbers)
        match = re.match(r"([a-zA-Z]+)([0-9]+)", file_name, re.I)
        if match:
            month_str, year_str = match.groups()
            # Abbreviating the month
            month_abbr = next((abbr for abbr, full in zip(calendar.month_abbr[1:], calendar.month_name[1:]) if month_str.lower() in full.lower()), month_str)

            # Process the input data frame using the MRC function.
            processed_dfs = MRC(file_name, dataframe)

            # Write each processed data frame to a different sheet.
            for sheet_name, processed_df in processed_dfs.items():
                # Use only the month abbreviation in the sheet name
                new_sheet_name = f"{month_abbr}_{sheet_name.split('_')[-1]}"
                processed_df.to_excel(writer, sheet_name=new_sheet_name, index=False)



In [ ]:
from google.colab import files

files.download('combined_mrc_sheets.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>